# Analyzing Rubik's Cube with Oscar

The following Julia session shows the computations from the [Rubik's Cube example](https://www.gap-system.org/Doc/Examples/rubik.html) from the GAP webpages.
A Julia session with equivalent computations using just the GAP interface [GAP.jl](https://github.com/oscar-system/GAP.jl) can be found inside [the documentation of this package](https://oscar-system.github.io/GAP.jl/stable).

First we load `Oscar.jl`.

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.11.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The OSCAR Development Team


We consider the group of transformations of Rubik's magic cube.
If we number the faces of this cube as follows

```
                     +--------------+
                     |              |
                     |  1    2    3 |
                     |              |
                     |  4  top    5 |
                     |              |
                     |  6    7    8 |
                     |              |
      +--------------+--------------+--------------+--------------+
      |              |              |              |              |
      |  9   10   11 | 17   18   19 | 25   26   27 | 33   34   35 |
      |              |              |              |              |
      | 12  left  13 | 20 front  21 | 28 right  29 | 36  rear  37 |
      |              |              |              |              |
      | 14   15   16 | 22   23   24 | 30   31   32 | 38   39   40 |
      |              |              |              |              |
      +--------------+--------------+--------------+--------------+
                     |              |
                     | 41   42   43 |
                     |              |
                     | 44 bottom 45 |
                     |              |
                     | 46   47   48 |
                     |              |
                     +--------------+
```

then the group is generated by the following generators, corresponding to the six faces of the cube.

In [2]:
g = symmetric_group(48);

l = [
     [[ 1, 3, 8, 6], [ 2, 5, 7, 4], [ 9,33,25,17], [10,34,26,18], [11,35,27,19]],
     [[ 9,11,16,14], [10,13,15,12], [ 1,17,41,40], [ 4,20,44,37], [ 6,22,46,35]],
     [[17,19,24,22], [18,21,23,20], [ 6,25,43,16], [ 7,28,42,13], [ 8,30,41,11]],
     [[25,27,32,30], [26,29,31,28], [ 3,38,43,19], [ 5,36,45,21], [ 8,33,48,24]],
     [[33,35,40,38], [34,37,39,36], [ 3, 9,46,32], [ 2,12,47,29], [ 1,14,48,27]],
     [[41,43,48,46], [42,45,47,44], [14,22,30,38], [15,23,31,39], [16,24,32,40]] ];

cube = sub(g, [cperm(g, x) for x in l])[1]

<permutation group with 6 generators>

First we want to know the order of this group.

In [3]:
cube_size = order(cube)

43252003274489856000

Since this is a little bit unhandy, let us factorize this number.

In [4]:
factor(cube_size)

1 * 5^3 * 7^2 * 2^27 * 11 * 3^14

(The result tells us that the order is 2<sup>27</sup> 3<sup>14</sup> 5<sup>3</sup> 7<sup>2</sup> 11.)

Next let us investigate the operation of the group on the 48 points.

In [5]:
orbs = orbits(cube);  map(collect, orbs)

2-element Vector{Vector{Int64}}:
 [1, 3, 17, 14, 8, 38, 9, 41, 19, 48  …  43, 11, 46, 40, 24, 27, 25, 35, 16, 32]
 [2, 5, 12, 7, 36, 10, 47, 4, 28, 45  …  20, 42, 26, 21, 37, 15, 31, 18, 23, 39]

The first orbit contains the points at the corners, the second those at the edges;
clearly the group cannot move a point at a corner onto a point at an edge.

So to investigate the cube group we first investigate the operation on the corner points.
Note that the constructed group that describes this operation will operate on the set `1:24`, not on the original set `orbs[1]`.

In [6]:
cube1 = image(action_homomorphism(orbs[1]))[1]

<permutation group with 6 generators>

In [7]:
degree(cube1)

24

In [8]:
order(cube1)

88179840

Now this group obviously operates transitively, but let us test whether it is also primitive.

In [9]:
corners = blocks(cube1);  collect(corners)

8-element Vector{Vector{Int64}}:
 [1, 7, 22]
 [2, 14, 20]
 [3, 12, 16]
 [4, 17, 18]
 [5, 9, 21]
 [6, 10, 24]
 [8, 11, 23]
 [13, 15, 19]

Those eight blocks correspond to the eight corners of the cube; on the one hand the group permutes those and on the other hand it permutes the three points at each corner cyclically.

In [10]:
blockhom1 = action_homomorphism(corners);  cube1b = image(blockhom1)[1]

Group([ (1,2,5,3), (1,3,7,4), (3,5,8,7), (2,6,8,5), (1,4,6,2), (4,7,8,6) ])

In [11]:
order(cube1b)

40320

Now a permutation group of degree 8 that has order 40320 must be the full symmetric group `Sym(8)` on eight points.

The next thing then is to investigate the kernel of this operation on blocks, i.e., the subgroup of `cube1` of those elements that fix the blocks setwise.

In [12]:
ker1 = kernel(blockhom1)[1]

<permutation group of size 2187 with 7 generators>

In [13]:
factor(order(ker1))

1 * 3^7

In [14]:
is_elementary_abelian(ker1)

true

We can show that the product of this elementary abelian group ``3^7`` with the `Sym(8)` is semidirect by finding a complement, i.e., a subgroup that has trivial intersection with the kernel and that generates `cube1` together with the kernel.

In [15]:
cmpl1 = complement_class_reps(cube1, ker1)

1-element Vector{PermGroup}:
 <permutation group of size 40320 with 7 generators>

In [16]:
cmpl1 = cmpl1[1]; order(cmpl1)

40320

We verify the complement properties:

In [17]:
order(intersect(cmpl1, ker1)[1])

1

In [18]:
sub(cube1, vcat(gens(cmpl1), gens(ker1)))[1] == cube1

true

There is even a more elegant way to show that `cmpl1` is a complement.

In [18]:
is_bijective(restrict_homomorphism(blockhom1, cmpl1))

true

Of course, theoretically it is clear that `cmpl1` must indeed be a complement.

In fact we know that `cube1` is a subgroup of index 3 in the wreath product of a cyclic 3 with `Sym(8)`.
This missing index 3 tells us that we do not have total freedom in turning the corners.
The following tests show that whenever we turn one corner clockwise we must turn another corner counterclockwise.

In [19]:
cperm([1, 7, 22]) in cube1

false

In [20]:
cperm([1, 7, 22], [2, 20, 14]) in cube1

true

More or less the same things happen when we consider the operation of the cube group on the edges.

In [21]:
cube2 = image(action_homomorphism(cube, orbs[2]))[1]

<permutation group with 6 generators>

In [22]:
degree(cube2)

24

In [23]:
order(cube2)

980995276800

In [24]:
edges = blocks(cube2);

In [25]:
collect(edges)

12-element Vector{Vector{Int64}}:
 [1, 11]
 [2, 17]
 [3, 19]
 [4, 22]
 [5, 13]
 [6, 8]
 [7, 24]
 [9, 18]
 [10, 21]
 [12, 15]
 [14, 20]
 [16, 23]

In [26]:
blockhom2 = action_homomorphism(edges)

Group homomorphism from 
<permutation group of size 980995276800 with 6 generators>
to
Sym( [ 1 .. 12 ] )

In [27]:
cube2b = image(blockhom2)[1]

Group([ (1,2,4,6), (3,6,10,11), (4,8,12,10), (2,5,9,8), (1,3,7,5), (7,11,12,9) ])

In [28]:
order(cube2b)

479001600

In [29]:
ker2 = kernel(blockhom2)[1]

<permutation group of size 2048 with 11 generators>

In [30]:
factor(order(ker2))

1 * 2^11

In [31]:
is_elementary_abelian(ker2)

true

In [32]:
cmpl2 = complement_class_reps(cube2, ker2)

4-element Vector{PermGroup}:
 <permutation group of size 479001600 with 11 generators>
 <permutation group of size 479001600 with 11 generators>
 <permutation group of size 479001600 with 11 generators>
 <permutation group of size 479001600 with 11 generators>

So there are even 4 classes of complements here.
This time we get a semidirect product of a ``2^11`` with an `Sym(12)`, namely a subgroup of index 2 of the wreath product of a cyclic 2 with `Sym(12)`.
Here the missing index 2 tells us again that we do not have total freedom in turning the edges.
The following tests show that whenever we flip one edge we must also flip another edge.

In [33]:
cperm([1, 11]) in cube2

false

In [34]:
cperm([1, 11], [2, 17]) in cube2

true

Since `cube1` and `cube2` are the groups describing the actions on the two orbits of `cube`, it is clear that `cube` is a subdirect product of those groups, i.e., a subgroup of the direct product. Comparing the sizes of `cube1`, `cube2`, and `cube` we see that `cube` must be a subgroup of index 2 in the direct product of those two groups.

In [35]:
order(cube)

43252003274489856000

In [36]:
order(cube1) * order(cube2)

86504006548979712000

This final missing index 2 tells us that we cannot operate on corners and edges totally independently.
The following tests show that whenever we exchange a pair of corners we must also exchange a pair of edges (and vice versa).

In [37]:
cflip = cperm([17, 19], [8, 11], [6, 25]);  cflip in cube

false

In [38]:
eflip = cperm([7, 28], [18, 21]);  eflip in cube

false

In [39]:
cflip * eflip in cube

true

As a last part of the structure analysis of the cube group let us compute the center of the cube group, i.e., the subgroup of those operations that can be performed either before or after any other operation with the same result.

In [40]:
z = center(cube)[1]

Group([ (2,34)(4,10)(5,26)(7,18)(12,37)(13,20)(15,44)(21,28)(23,42)(29,36)(31,45)(39,47) ])

We see that the centre contains one nontrivial element, namely the operation that flips all 12 edges simultaneously.

Finally we turn to the original idea connected with the cube, namely to find a sequence of turns of the faces that will transform the cube back into its original state. This amounts to a decomposition of a given element of the cube group into a product of the generators. For this purpose we introduce a free group and a homomorphism of it onto the cube group.

(In order to make the results of the decompositions reproducible, we reset the random number generator that is involved in the computations.)

In [41]:
GAP.Globals.Reset(GAP.Globals.GlobalMersenneTwister);

In [42]:
f = free_group(["t", "l", "f", "r", "e", "b"])

<free group on the generators [ t, l, f, r, e, b ]>

In [43]:
fhom = hom(f, cube, gens(cube))

Group homomorphism from 
<free group on the generators [ t, l, f, r, e, b ]>
to
<permutation group of size 43252003274489856000 with 6 generators>

Using this homomorphism, we can now decompose elements into generators. The method used utilizes a stabilizer chain and does not enumerate all group elements, therefore the words obtained are not the shortest possible, though they are short enough for hand solutions.

(In order to make the results of the decompositions reproducible, we reset the random number generator that is involved in the computations.)
First we decompose the nonidentity centre element:

In [44]:
zgen = gen(z, 1);  pre1 = haspreimage(fhom, zgen)[2]

t^-1*l*t*l^-1*e^-1*t*f^-1*e*l*f*(t^-1*l^-1)^2*t*l*f*t*r*t^-1*r^-1*f^-1*l^-1*e^-1*t^-1*e*t*l*t*f*t*r*t^-1*r^-1*f^-1*t^-2*f*t^-1*f^-1*t^2*e*l^-1*e^-1*t^-1*l*t*l*f*t^-1*f^-1*l^-1*t*l*t^2*l^-1*t^-1*l*t^-1*l^-1*t^2*f^-1*l^-1*t^-1*l*t*f*t*l^-1*t*l*f^-1*l*f*l^-1*t^-2*l^-1*f^-1*l*f*l*t*e^-1*t*e*l*t*l^-1*e*l*e^-1*t^-1*f*l*f*l^-1*f^-1*t^-2*f^-1*t*l^-1*f*t^-2*f^-2*b*r^-1*b^-1*t^-2*e^-1*r^-2*e*t^-1*r*b*e^-1*b^-1*l^-1*r^-2*t^-2*l^-1*b^-1*r^-1*e^-1

In [45]:
length(pre1)

134

Next we decompose some element arbitrarily chosen by us:

In [46]:
pre2 = haspreimage(fhom, cflip * eflip)[2]

l^-1*t^-1*l*f*r*t*r^-1*f^-1*l*t*f*t^-1*f^-1*l^-1*t^2*f*t*l*t*l^-1*f^-1*l*t^-1*l^-1*f*t^-1*f^-1*l*t*l^-1*t*l*t^-2*l^-1*f*(t*r*t^-1*r^-1)^2*f^-1*t*l*f^-1*l^-1*f*l^-1*t^-1*l*t^-2*f*t*(f^-1*l^-1)^2*l^-1*f*l*e^-1*t*e*l*t^-1*e^-1*t^-1*e*l*b*f^-1*b^-1

In [47]:
length(pre2)

77

Last we let GAP choose a random element ...

In [48]:
r = rand(cube)

(1,3,9,33,35,27)(2,13,39,26,23,7,15)(5,42,18,44,34,20,47)(6,16,8,30,14,11,41,19,24,40,17,22,25,43,46)(12,31,37,45)(29,36)

In [49]:
pre3 = haspreimage(fhom, r)[2]

l*b^-1*f^-1*t^-1*r*t^-1*e^-1*r^-2*e*t^-1*b^-1*e^-1*b*f^2*t*f^-1*l*t^-1*(f*l)^2*f^-1*l^-1*f^-1*t*f^-1*l*f*t*e^-1*t*e*t^2*l^-1*f^-1*l^-1*f*e^-1*t^-1*e*l*f^-1*l*f*l^-1*t^-1*f*r*t*r^-1*t^-1*f^-1*t*l*t*l^-1*t^-1*l^-1*e*l*e^-1*l*f*t*f^-1*t^-1*l^-1*f*r*t^-1*r^-1*f^-1*l^-1*t*l

In [50]:
length(pre3)

79

... and we verify that the decomposition is correct:

In [51]:
fhom(pre3) == r

true

This concludes our example.